In [1]:
from cgeniepy.foram import ForamModel as fm
import pandas as pd
from cgeniepy.grid import GENIE_lat
import numpy as np
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

foram = ["bn", "bs", "sn", "ss"]

lat = GENIE_lat() * -1
lat_array = np.broadcast_to(lat, (36, 36)).T


mod_paths = [
              "../model/muffin.CBE.worlg4.BASESFeTDTL.SPIN",
              "../model/muffin.CBE.GIteiiva.BASESFeTDTL_rb.SPIN",
            "../model/muffin.CBE.worlg4.BASESFeTDTL.historical",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.1p5deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.2deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.3deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.4deg",
             ]

export_files= ["../data/model_drived/pi_foramecogenie.csv",
               "../data/model_drived/lgm_foramecogenie.csv",
               "../data/model_drived/present_foramecogenie.csv",
               "../data/model_drived/future1p5_foramecogenie.csv",
                "../data/model_drived/future2_foramecogenie.csv",
                "../data/model_drived/future3_foramecogenie.csv",
                "../data/model_drived/future4_foramecogenie.csv",
               ]


for path, csv_name in zip(mod_paths, export_files):
    mod = fm(path)
    bn = mod.select_foramtype("bn").biomass().isel(time=-1).array.values.ravel()
    bs = mod.select_foramtype("bs").biomass().isel(time=-1).array.values.ravel()
    sn = mod.select_foramtype("sn").biomass().isel(time=-1).array.values.ravel()
    ss = mod.select_foramtype("ss").biomass().isel(time=-1).array.values.ravel()

    sst = mod.select_var("ocn_sur_temp").isel(time=-1).array.values.ravel()
    sal = mod.select_var("ocn_sur_sal").isel(time=-1).array.values.ravel()
    po4 = mod.select_var("ocn_sur_PO4").isel(time=-1).array.values.ravel()
    light = mod.select_var("phys_fxsw").isel(time=-1).array.values.ravel()
    chl_total = mod.select_var("eco2D_Plankton_C_Total").isel(time=-1).array.values.ravel()
    prey = mod.select_var("eco2D_Plankton_C_004").isel(time=-1).array.values.ravel()
    fe = mod.select_var("ocn_sur_TDFe").isel(time=-1).array.values.ravel()
    mld = mod.select_var("phys_MLD").isel(time=-1).array.values.ravel()

    # masked!
    #random_var = mod.select_var("ocn_sur_temp").isel(time=-1).array.values
    #mask = np.where(np.isnan(random_var), np.nan, 1)
    ## masked lat
    #masked_lat = lat_array * mask


    #phys_w = mod.select_var("phys_w").isel(time=-1, zt=1).array
    #upwelling_discrete = np.where(phys_w >0, 1, 0)

    # temperature gradient between 283 m and 80 m
    therm_gradient = mod.select_var("phys_w").isel(time=-1, zt=3).array - mod.select_var("phys_w").isel(time=-1, zt=1).array

    df = pd.DataFrame({'bn': bn,
                       'bs': bs,
                       'sn':sn,
                       'ss':ss,
                       'sst': sst,
                       'sal': sal,
                       'po4': po4,
                       'prey': prey,
                       'chl_total': chl_total,
                       'fe': fe,
                       'mld': mld,
                       'light':light,
                       #"lat": masked_lat.ravel(),
                       #"upwelling": upwelling_discrete.ravel(),
                       "stratification": therm_gradient.values.ravel()
                       })

    # drop na
    df = df.dropna()
    #df = df.query('bn > 0 & bs > 0 & sn > 0 & ss > 0')
    #df = df.query('bn < 1 & bs < 1 & sn < 1 & ss < 1')
    df.to_csv(csv_name)